In [ ]:
# %%writefile Ind_Behavior.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tools.Sample_Tools as smpl

from base.JuUnits import excute_for_multidates

from sklearn import linear_model


In [ ]:
def cal_ret_market(market_value,ret_excess_data):
    '''计算市场（平均）收益
        :param market_value：{pd.Series} --市值
        :param ret_excess_data：{pd.Series} --超额回报
        
        :return: {pd.Series}  -- 市值加权的市场平均收益
    '''
    ##  不取对数有时候有精度问题,权重不是精确1
    market_value_log = np.log(market_value)
    weight = market_value_log / market_value_log.sum()
    ret_market_f = (ret_excess_data * weight).sum()
    return ret_market_f
'''
    需要准备的基础数据
    stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    smpl.optimize_data_type(stock_df)
    # stock_df = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()

    ## 日无风险回报
    ret_fs_data = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
    ret_fs_data = (ret_fs_data['high'].astype(np.float32)+ret_fs_data['low'].astype(np.float32))/2 * 0.01
    ret_fs_daily = ret_fs_data/252
'''

def prepare_data(stock_df,ret_fs):
    '''数据准备
        :param stock_df：{pd.DataFrame} --股票数据，包含close,volume,market_value，liquidity_market_value,industry
        :param ret_fs：{pd.Series} --无风险回报
        
        :return: {set in [ret_t,ret_t_excess,market_value_t,ret_excess_market_t]}
    '''

    ret_t = smpl.get_current_return(stock_df,'close')

    # 超额回报
    ret_t_excess = ret_t.groupby(pd.Grouper(level='date', freq='1M')).apply(
            lambda x:(x-ret_fs.get(x.index[0][0].strftime('%Y-%m'),default=ret_fs[-1])))

    # 市值
    market_value_t = stock_df['market_value']

    # 市场收益，日内全市场收益加权平均
    ret_excess_market_t = excute_for_multidates(ret_t_excess,
                                         lambda ret: cal_ret_market(market_value_t.loc[ret.index[0][0]],ret), 
                                         level=0)
    
    return ret_t, ret_t_excess, market_value_t, ret_excess_market_t

In [ ]:
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)
# stock_df = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()

## 日无风险回报
ret_fs_data = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs_data = (ret_fs_data['high'].astype(np.float32)+ret_fs_data['low'].astype(np.float32))/2 * 0.01
ret_fs_daily = ret_fs_data/252


ret_t, ret_t_excess, market_value_t, ret_excess_market_t = prepare_data(stock_df,ret_fs_daily)



In [ ]:
def sizelg(stock_data):
    '''市值因子
        :param stock_data：{pd.DataFrame} --需要包含market_value
    '''
    mv = np.log(stock_data['market_value'])
    mv.name = 'sizelg'
    return mv

def bp(stock_data):
    '''Book-to-Price
        :param stock_data：{pd.DataFrame} --需要包含close
    '''
    data = smpl.add_report_inds(stock_data[['close']],'netAssetsPerShare')
    bp = data['close']/data['netAssetsPerShare']
    bp.name = 'bp'
    return bp

In [ ]:
# ret_exce = alpha + b_1*(ret_mk_exce)+b_2*SMB+b_3*HML + epsilon
# 市场收益率、市值因子收益率、估值因子收益率
# 因子收益率为样本空间内因子值最小的 1/3 股票构建的流通市值加权组合与最大 1/3 股票组合收益率之差。
# ivol = epsilon.std(ddof=1) * sqrt(N) 
# SMB

# HML

# r2 = 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2

# 特异度 1-r2 
# 特质波动=sqrt（特异度）*总波动率 ？？？

# ln(turnover) = alpha + b* ln(mv)+epsilon
# 回归残差项我们用来作为剔除市值后的换手率代理变量，我们称之为流通市值调整换手。


In [ ]:
# behaviorIndex = 1/2*[Q(IVR)+Q(adjTurnover)]
# behaviorIndex为股票i在时刻t的交易热度，IVR为股票i在时刻t的特异度，adjTurnover为股票i在时刻t的市值调整换手。
# Q表示股票i的指标在时刻t样本空间内所有股票中所对应的分位数（累计分布概率）。
# 根据交易热度的定义可知，交易热度的取值在0-1之间，交易热度取值越高，表明股票交易的活跃程度越高，反应的投机程度越大，后期预期收益率越低。